In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as f
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from sklearn import preprocessing
from tqdm import tqdm
from sklearn.preprocessing import FunctionTransformer
import seaborn as sns 
from scipy import stats
import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
import h2o
from h2o.automl import H2OAutoML
from sklearn.neural_network import MLPRegressor
from scipy.stats import norm
import copy
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor, Pool, metrics, cv
import xgboost as xgb
from scipy.stats import gmean
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder


In [44]:
train_df = pd.read_csv('./new_train.csv', parse_dates=["startdate"])
test_df = pd.read_csv('./new_test.csv', parse_dates=["startdate"])

In [46]:
_train = train_df
_test = test_df

In [45]:
exclude_cols = ['index', 'startdate']
temporal_attrs = ['year', 'month', 'day_of_year', 'season', 'day_of_year_sin', 'day_of_year_cos','month_sin', 'month_cos', 'season_sin', 'season_cos']
loc_attrs = ['lat', 'lon', 'loc_group']
embedding_attrs = ['climateregions__climateregion']
target=["contest-tmp2m-14d__tmp2m"]
main_attrs = [c for c in train_df.columns if c not in exclude_cols and c not in temporal_attrs and c not in loc_attrs and c not in target and c not in embedding_attrs]
print("Main features:", len(main_attrs))

Main features: 240


In [47]:
train_df['climateregions__climateregion'].value_counts()

1     139621
9      52632
4      51901
7      40936
8      22661
3      13889
10     12427
2       9503
6       9503
11      8041
0       5117
5       4386
12      2924
13      1462
14       731
Name: climateregions__climateregion, dtype: int64

In [51]:
train_df = train_df[train_df['climateregions__climateregion'] == 14]
test_df = test_df[test_df['climateregions__climateregion'] == 14]

In [49]:
best_cols = ['contest-wind-h500-14d__wind-hgt-500',
 'contest-slp-14d__slp',
 'nmme-tmp2m-34w__ccsm3',
 'elevation__elevation',
 'lon',
 'contest-prwtr-eatm-14d__prwtr',
 'lat',
 'climateregions__climateregion',
 'contest-wind-h100-14d__wind-hgt-100',
 'contest-pres-sfc-gauss-14d__pres',
 'season_sin',
 'day_of_year_sin',
 'contest-precip-14d__precip',
 'contest-wind-uwnd-250-14d__wind-uwnd-250',
 'nmme-prate-34w__cfsv2',
 'nmme-prate-34w__nasa',
 'nmme-prate-56w__gfdlflora',
 'wind-uwnd-250-2010-7',
 'contest-wind-vwnd-925-14d__wind-vwnd-925',
 'nmme-prate-34w__nmmemean',
 'nmme0-prate-34w__ccsm30',
 'contest-wind-h850-14d__wind-hgt-850',
 'contest-wind-uwnd-925-14d__wind-uwnd-925',
 'nmme0-prate-56w__cfsv20',
 'month_sin',
 'nmme-prate-34w__cancm3',
 'contest-rhum-sig995-14d__rhum',
 'nmme-prate-34w__gfdlflorb',
 'wind-hgt-850-2010-4',
 'contest-wind-vwnd-250-14d__wind-vwnd-250',
 'wind-hgt-100-2010-2',
 'wind-uwnd-250-2010-18',
 'wind-hgt-10-2010-5',
 'wind-uwnd-250-2010-15',
 'wind-uwnd-250-2010-4',
 'nmme0-prate-56w__nasa0',
 'nmme0-prate-34w__cfsv20',
 'wind-vwnd-250-2010-10',
 'contest-wind-h10-14d__wind-hgt-10',
 'wind-uwnd-925-2010-15',
 'wind-vwnd-250-2010-3',
 'nmme-prate-34w__cancm4',
 'sst-2010-4',
 'nmme0-prate-56w__ccsm30',
 'wind-uwnd-250-2010-16',
 'nmme0-prate-34w__gfdl0',
 'nmme0-prate-56w__cancm40',
 'sst-2010-1',
 'sst-2010-3',
 'wind-uwnd-250-2010-14',
 'nmme0-prate-34w__gfdlflora0',
 'nmme-prate-34w__gfdl',
 'wind-hgt-850-2010-9',
 'wind-vwnd-250-2010-1',
 'nmme-prate-34w__gfdlflora',
 'sst-2010-5',
 'cancm30',
 'nmme-prate-34w__ccsm4',
 'nmme0-prate-34w__nasa0',
 'wind-hgt-500-2010-9',
 'nmme0-prate-34w__cancm30',
 'wind-vwnd-250-2010-13']

In [50]:
train_independent_corr = train_df[best_cols].corr()
train_seleted_corr_columns = np.full((train_independent_corr.shape[0],), True, dtype=bool)
for i in range(train_independent_corr.shape[0]):
    for j in range(i + 1, train_independent_corr.shape[0]):
        if train_independent_corr.iloc[i, j] >= 0.95:
            
            if train_seleted_corr_columns[j]:
                train_seleted_corr_columns[j] = False
train_selected_columns = train_df[best_cols].columns[train_seleted_corr_columns]
print(train_selected_columns.shape)

(58,)


In [52]:
X = train_df[train_selected_columns].values
y = train_df[target[0]].values

skf = KFold(n_splits=5, random_state=2048, shuffle=True)
skf.get_n_splits(X, y)

train_index_list = []
test_index_list = []
train_x_list = []
val_x_list = []
train_y_list = []
val_y_list = []

for i, (train_index, test_index) in tqdm(enumerate(skf.split(X, y))):
    train_index_list.append(train_index)
    test_index_list.append(test_index)

    train_x_list.append(X[train_index, :])
    val_x_list.append(X[test_index, :])
    train_y_list.append(y[train_index])
    val_y_list.append(y[test_index])    
def train_catboost(x, y):
    print(x.shape, y.shape)
    
    cat_models = []
    for i in tqdm(range(5)):
        train_x, val_x = train_x_list[i], val_x_list[i]
        train_y, val_y = train_y_list[i], val_y_list[i]
        clf = CatBoostRegressor(iterations=1000, verbose=200
#                                 , task_type="GPU",
#                            devices='0:1',
#                                 ,learning_rate=0.5
                               )
        clf.fit(train_x, train_y, eval_set=[(val_x, val_y)], verbose=True, early_stopping_rounds=100)
        cat_models.append(clf)
    
    return cat_models
cat_models = train_catboost(X, y)


5it [00:00, 3019.22it/s]


(731, 58) (731,)


  0%|                                                                                                | 0/5 [00:00<?, ?it/s]

Learning rate set to 0.046797
0:	learn: 9.8364423	test: 9.7343926	best: 9.7343926 (0)	total: 3.92ms	remaining: 3.92s
1:	learn: 9.4549316	test: 9.3551619	best: 9.3551619 (1)	total: 5.9ms	remaining: 2.94s
2:	learn: 9.0847366	test: 8.9862567	best: 8.9862567 (2)	total: 7.84ms	remaining: 2.6s
3:	learn: 8.7482819	test: 8.6477998	best: 8.6477998 (3)	total: 9.76ms	remaining: 2.43s
4:	learn: 8.4022397	test: 8.3033246	best: 8.3033246 (4)	total: 11.6ms	remaining: 2.31s
5:	learn: 8.0805808	test: 7.9797188	best: 7.9797188 (5)	total: 13.7ms	remaining: 2.27s
6:	learn: 7.7911899	test: 7.6903096	best: 7.6903096 (6)	total: 15.6ms	remaining: 2.21s
7:	learn: 7.4815629	test: 7.3854638	best: 7.3854638 (7)	total: 18ms	remaining: 2.23s
8:	learn: 7.1931216	test: 7.1053384	best: 7.1053384 (8)	total: 19.9ms	remaining: 2.19s
9:	learn: 6.9214405	test: 6.8284136	best: 6.8284136 (9)	total: 21.9ms	remaining: 2.16s
10:	learn: 6.6525026	test: 6.5621234	best: 6.5621234 (10)	total: 24.3ms	remaining: 2.19s
11:	learn: 6.39

94:	learn: 0.6584743	test: 0.7911611	best: 0.7911611 (94)	total: 192ms	remaining: 1.83s
95:	learn: 0.6501448	test: 0.7845967	best: 0.7845967 (95)	total: 197ms	remaining: 1.85s
96:	learn: 0.6421034	test: 0.7758401	best: 0.7758401 (96)	total: 199ms	remaining: 1.85s
97:	learn: 0.6342653	test: 0.7685598	best: 0.7685598 (97)	total: 201ms	remaining: 1.85s
98:	learn: 0.6263777	test: 0.7595652	best: 0.7595652 (98)	total: 203ms	remaining: 1.84s
99:	learn: 0.6198347	test: 0.7528434	best: 0.7528434 (99)	total: 205ms	remaining: 1.84s
100:	learn: 0.6124768	test: 0.7460763	best: 0.7460763 (100)	total: 207ms	remaining: 1.84s
101:	learn: 0.6064444	test: 0.7390507	best: 0.7390507 (101)	total: 209ms	remaining: 1.84s
102:	learn: 0.5978767	test: 0.7306419	best: 0.7306419 (102)	total: 211ms	remaining: 1.84s
103:	learn: 0.5908625	test: 0.7242980	best: 0.7242980 (103)	total: 213ms	remaining: 1.83s
104:	learn: 0.5850725	test: 0.7192726	best: 0.7192726 (104)	total: 215ms	remaining: 1.83s
105:	learn: 0.5796078	

...

693:	learn: 0.1144265	test: 0.3981571	best: 0.3979560 (690)	total: 1.36s	remaining: 601ms
694:	learn: 0.1143945	test: 0.3981742	best: 0.3979560 (690)	total: 1.37s	remaining: 600ms
695:	learn: 0.1142567	test: 0.3982244	best: 0.3979560 (690)	total: 1.37s	remaining: 598ms
696:	learn: 0.1140313	test: 0.3980236	best: 0.3979560 (690)	total: 1.37s	remaining: 596ms
697:	learn: 0.1139787	test: 0.3980598	best: 0.3979560 (690)	total: 1.37s	remaining: 594ms
698:	learn: 0.1139520	test: 0.3980996	best: 0.3979560 (690)	total: 1.37s	remaining: 592ms
699:	learn: 0.1138900	test: 0.3980904	best: 0.3979560 (690)	total: 1.38s	remaining: 590ms
700:	learn: 0.1137450	test: 0.3980258	best: 0.3979560 (690)	total: 1.38s	remaining: 588ms
701:	learn: 0.1135655	test: 0.3979283	best: 0.3979283 (701)	total: 1.38s	remaining: 586ms
702:	learn: 0.1135375	test: 0.3979294	best: 0.3979283 (701)	total: 1.38s	remaining: 584ms
703:	learn: 0.1132699	test: 0.3978296	best: 0.3978296 (703)	total: 1.38s	remaining: 582ms
704:	learn

790:	learn: 0.0977568	test: 0.3915000	best: 0.3915000 (790)	total: 1.56s	remaining: 412ms
791:	learn: 0.0976175	test: 0.3913330	best: 0.3913330 (791)	total: 1.56s	remaining: 410ms
792:	learn: 0.0974241	test: 0.3911606	best: 0.3911606 (792)	total: 1.56s	remaining: 408ms
793:	learn: 0.0973225	test: 0.3911148	best: 0.3911148 (793)	total: 1.56s	remaining: 406ms
794:	learn: 0.0971338	test: 0.3910357	best: 0.3910357 (794)	total: 1.56s	remaining: 403ms
795:	learn: 0.0971150	test: 0.3910407	best: 0.3910357 (794)	total: 1.57s	remaining: 401ms
796:	learn: 0.0969956	test: 0.3909006	best: 0.3909006 (796)	total: 1.57s	remaining: 399ms
797:	learn: 0.0969687	test: 0.3908706	best: 0.3908706 (797)	total: 1.57s	remaining: 397ms
798:	learn: 0.0968144	test: 0.3907942	best: 0.3907942 (798)	total: 1.57s	remaining: 395ms
799:	learn: 0.0965814	test: 0.3907692	best: 0.3907692 (799)	total: 1.57s	remaining: 393ms
800:	learn: 0.0964417	test: 0.3907287	best: 0.3907287 (800)	total: 1.57s	remaining: 391ms
801:	learn

889:	learn: 0.0841299	test: 0.3862846	best: 0.3862521 (888)	total: 1.75s	remaining: 217ms
890:	learn: 0.0839451	test: 0.3861558	best: 0.3861558 (890)	total: 1.75s	remaining: 215ms
891:	learn: 0.0837951	test: 0.3860013	best: 0.3860013 (891)	total: 1.76s	remaining: 213ms
892:	learn: 0.0836380	test: 0.3859197	best: 0.3859197 (892)	total: 1.76s	remaining: 211ms
893:	learn: 0.0834193	test: 0.3858870	best: 0.3858870 (893)	total: 1.76s	remaining: 209ms
894:	learn: 0.0832645	test: 0.3857300	best: 0.3857300 (894)	total: 1.76s	remaining: 207ms
895:	learn: 0.0831261	test: 0.3855163	best: 0.3855163 (895)	total: 1.76s	remaining: 205ms
896:	learn: 0.0829817	test: 0.3854931	best: 0.3854931 (896)	total: 1.77s	remaining: 203ms
897:	learn: 0.0828798	test: 0.3854761	best: 0.3854761 (897)	total: 1.77s	remaining: 201ms
898:	learn: 0.0827223	test: 0.3854549	best: 0.3854549 (898)	total: 1.77s	remaining: 199ms
899:	learn: 0.0825817	test: 0.3853079	best: 0.3853079 (899)	total: 1.77s	remaining: 197ms
900:	learn

987:	learn: 0.0726477	test: 0.3815019	best: 0.3815019 (987)	total: 1.95s	remaining: 23.7ms
988:	learn: 0.0724979	test: 0.3814707	best: 0.3814707 (988)	total: 1.95s	remaining: 21.7ms
989:	learn: 0.0723868	test: 0.3814578	best: 0.3814578 (989)	total: 1.95s	remaining: 19.7ms
990:	learn: 0.0722150	test: 0.3813363	best: 0.3813363 (990)	total: 1.95s	remaining: 17.7ms
991:	learn: 0.0720382	test: 0.3811575	best: 0.3811575 (991)	total: 1.96s	remaining: 15.8ms
992:	learn: 0.0720169	test: 0.3811518	best: 0.3811518 (992)	total: 1.96s	remaining: 13.8ms
993:	learn: 0.0718551	test: 0.3810492	best: 0.3810492 (993)	total: 1.96s	remaining: 11.8ms
994:	learn: 0.0717554	test: 0.3810153	best: 0.3810153 (994)	total: 1.96s	remaining: 9.86ms
995:	learn: 0.0717371	test: 0.3809992	best: 0.3809992 (995)	total: 1.96s	remaining: 7.88ms
996:	learn: 0.0715803	test: 0.3809214	best: 0.3809214 (996)	total: 1.97s	remaining: 5.91ms
997:	learn: 0.0714535	test: 0.3808899	best: 0.3808899 (997)	total: 1.97s	remaining: 3.94ms

100%|████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:12<00:00,  2.45s/it]


In [53]:
train_preds = []
for cat_model in tqdm(cat_models):
    train_preds.append(cat_model.predict(train_df[train_selected_columns].values))
train_s = 0
for train_red in train_preds:
    train_s += train_red
train_s = train_s/5

100%|███████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 211.73it/s]


In [56]:
cnt = 0
for i in range(len(train_s)):
    residual = np.abs(train_s[i] - train_df[target[0]].tolist()[i])
    if (residual > 0.5):
        cnt += 1
cnt / len(train_s)

0.0

In [57]:
preds = []
for cat_model in tqdm(cat_models):
    preds.append(cat_model.predict(test_df[train_selected_columns].values))
s = 0
for pred in preds:
    s += pred
s = s/5

100%|███████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 393.66it/s]


In [58]:
output_df = pd.DataFrame(columns = ['contest-tmp2m-14d__tmp2m', 'index'])
output_df['contest-tmp2m-14d__tmp2m'] = s
output_df['index'] = test_df['index'].tolist()

In [59]:
best_df = pd.read_csv('best_sub.csv')
y_best = best_df[target[0]].values
merged_df = best_df.merge(output_df, how='left', on='index')
merged_df['contest-tmp2m-14d__tmp2m_y'].isnull().sum()
import math
t = merged_df.values
for i in range(t.shape[0]):
    if not math.isnan(t[i][2]):
        print(t[i][0] - t[i][2])
import math
def preprocess_result(x):
    r1, r2 = x
    if math.isnan(r2):
        return r1
    return r2 * 0.3 + r1 * 0.7
result = merged_df[['contest-tmp2m-14d__tmp2m_x', 'contest-tmp2m-14d__tmp2m_y']].apply(preprocess_result, axis=1)
df = pd.read_csv('./sample_solution.csv')
df['contest-tmp2m-14d__tmp2m'] = result
df.to_csv('submission_region_14_v1.csv', index=False)

from numpy import dot
from numpy.linalg import norm
y_best = best_df[target[0]].values
cos_sim = dot(y_best, result)/(norm(y_best)*norm(result))
print("cos_sim with best submission:", cos_sim)

-0.5586540298359388
-0.7302318726494441
-0.4568159774935854
-0.12034210974901782
-0.3993808508932517
-0.1573127407182966
-0.13886350913237955
-0.32862832228830463
-0.3162123341769689
-0.42130427292211614
-0.31790312173073687
-0.10099126757452126
0.03340023818719473
-0.11490068073451098
0.4248508885629487
0.008118092845354141
0.10957032137838318
-0.015514673948803903
-0.12089371560467654
0.10254697598123386
-0.21802783226799605
-0.14217709224102926
0.19253810274638639
-0.07462875976770711
-0.0479271166544919
0.046901564846985266
-0.1818949894712567
-0.4834604208428015
-0.03801546417134105
0.2919510364583999
0.2761150710669007
0.313781757899811
0.2693537765814984
-0.013041672094749046
0.39733865507655963
0.4266839444539512
0.8015256857820177
0.6692051686777436
0.867755022271246
0.8633917204429791
0.8627162823252439
1.1166997760309574
1.4257573528828011
1.4259283086085492
1.4396158827686634
0.8526573452330357
1.2493733559716338
1.4580871726243991
1.6490718274539784
1.7752142336258094
2.23